##Set up


In [15]:
# lots of imports for now. Will strip it down later

import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report, plot_roc_curve, roc_auc_score, plot_confusion_matrix
from sklearn.inspection import permutation_importance

from xgboost import XGBRegressor, XGBClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
#from sklearn.

#from google.colab import files
#from google.colab import drive
#drive.mount('drive', force_remount=True)

In [13]:
# data_path references data.csv from our github repo
data_path = 'https://raw.githubusercontent.com/StephenSpicer/Spotify_Music_Discovery_LS_DS_BW/main/data/data.csv'

## Wrangle Data


In [14]:
# Very basic function. Need to add new features if necessary.

# Data Wrangle function:


def wrangle(filepath):
  df = pd.read_csv(filepath, parse_dates=['release_date'])


  #make new features
  df['month'] = df['release_date'].dt.month


  #set release date as the index
  df.set_index('release_date', inplace=True)


  #rearraging similar features to be together to improve readability
  df = df[['id', 'artists', 'name', 'year', 'month', 'duration_ms', 'popularity',
           'valence', 'acousticness', 'danceability',
           'energy', 'instrumentalness', 'liveness', 'loudness',  
           'speechiness','tempo', 'key',
           'explicit', 'mode']]
           
  return df



In [17]:
# read in the data and save it to df
df = wrangle(data_path)

## EDA


In [20]:
df.head(10)

,id,artists,name,year,month,duration_ms,popularity,valence,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,key,explicit,mode
release_date,,,,,,,,,,,,,,,,,,,
1920-01-01,0cS0A1fUEUd1EW3FcF8AEI,['Mamie Smith'],Keep A Song In Your Soul,1920,1,168333,12,0.6340,0.991000,0.598,0.2240,0.000522,0.3790,-12.628,0.0936,149.976,5,0,0
1920-01-05,0hbkKFIJm7Z05H8Zl9w30f,"[""Screamin' Jay Hawkins""]",I Put A Spell On You,1920,1,150200,7,0.9500,0.643000,0.852,0.5170,0.026400,0.0809,-7.261,0.0534,86.889,5,0,0
1920-01-01,11m7laMUgmOKqI3oYzuhne,['Mamie Smith'],Golfing Papa,1920,1,163827,4,0.6890,0.993000,0.647,0.1860,0.000018,0.5190,-12.098,0.1740,97.600,0,0,1
1920-01-01,19Lc5SfJJ5O1oaxY0fpwfh,['Oscar Velazquez'],True House Music - Xavier Santos & Carlos Gomi...,1920,1,422087,17,0.0422,0.000173,0.730,0.7980,0.801000,0.1280,-7.311,0.0425,127.997,2,0,1
1920-10-01,2hJjbsLCytGsnAHfdsLejp,['Mixe'],Xuniverxe,1920,10,165224,2,0.2990,0.295000,0.704,0.7070,0.000246,0.4020,-6.036,0.0768,122.076,10,1,0
1920-01-01,3HnrHGLE9u2MjHtdobfWl9,['Mamie Smith & Her Jazz Hounds'],Crazy Blues - 78rpm Version,1920,1,198627,9,0.4770,0.996000,0.424,0.2450,0.799000,0.2350,-11.470,0.0397,103.870,5,0,1
1920-01-01,5DlCyqLyX2AOVDTjjkDZ8x,['Mamie Smith'],Don't You Advertise Your Man,1920,1,195200,5,0.4870,0.992000,0.782,0.0573,0.000002,0.1760,-12.453,0.0592,85.652,5,0,1
1920-01-01,02FzJbHtqElixxCmrpSCUa,['Mamie Smith & Her Jazz Hounds'],Arkansas Blues,1920,1,186173,0,0.3660,0.996000,0.474,0.2390,0.186000,0.1950,-9.712,0.0289,78.784,9,0,1
1920-07-08,02i59gYdjlhBmbbWhf8YuK,['Francisco Canaro'],La Chacarera - Remasterizado,1920,7,146840,0,0.6210,0.996000,0.469,0.2380,0.960000,0.1490,-18.717,0.0741,130.060,8,0,1


In [21]:
df.columns

Index(['id', 'artists', 'name', 'year', 'month', 'duration_ms', 'popularity',
       'valence', 'acousticness', 'danceability', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'key', 'explicit',
       'mode'],
      dtype='object')

In [23]:
df.shape

(174389, 19)

In [24]:
df.nunique()

id                  172230
artists              36195
name                137013
year                   102
month                   12
duration_ms          56306
popularity              98
valence               1707
acousticness          4929
danceability          1233
energy                2306
instrumentalness      5400
liveness              1740
loudness             25580
speechiness           1633
tempo                84123
key                     12
explicit                 2
mode                     2
dtype: int64

## Split Data


## Establish Baseline

##Build Model

##Check Metrics

##Tune Model

## Communicate Results